In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

# Leer data de train y test
file_path = "../datos-estadistica/datoscsv.csv"
df = pd.read_csv(file_path)

df_train = df[~df['City'].str.contains('test')].copy()
df_test = df[df['City'].str.contains('test')].copy()

# Escalar los datos
scaler = StandardScaler()
df_train_scaled = scaler.fit_transform(df_train.drop('City', axis=1))
df_test_scaled = scaler.transform(df_test.drop('City', axis=1))

def initialize_centroids(df, k):
    """Inicializa los centroides seleccionando k puntos aleatorios de los datos"""
    return df[np.random.choice(df.shape[0], k, replace=False)]

def assign_clusters(df, centroids):
    """Asigna cada punto al cluster más cercano"""
    distances = cdist(df, centroids, 'euclidean')
    return np.argmin(distances, axis=1)

def update_centroids(df, labels, k):
    """Recalcula los centroides como la media de todos los puntos asignados a cada cluster"""
    new_centroids = np.zeros((k, df.shape[1]))
    for i in range(k):
        points_in_cluster = df[labels == i]
        new_centroids[i, :] = points_in_cluster.mean(axis=0)
    return new_centroids

def kmeans(df, k, max_iters=100, tol=1e-4):
    """Implementación del algoritmo K-means desde cero"""
    centroids = initialize_centroids(df, k)
    
    for _ in range(max_iters):
        labels = assign_clusters(df, centroids)
        new_centroids = update_centroids(df, labels, k)
        
        # Verificar la convergencia (si los centroides no cambian mucho)
        if np.all(np.abs(new_centroids - centroids) < tol):
            break
        
        centroids = new_centroids
    
    return centroids, labels

# Ejecutar K-means y asignar clusters a los datos de entrenamiento y prueba
centroids, labels = kmeans(df_train_scaled, 6)
df_train['Cluster'] = labels
df_test['Cluster_Predicted'] = assign_clusters(df_test_scaled, centroids)

# Encontrar las ciudades más cercanas en el conjunto de entrenamiento
closest_cities = []
for idx, row in enumerate(df_test.iterrows()):  # Usar un índice separado `idx`
    cluster_id = row[1]['Cluster_Predicted']
    
    # Filtrar las ciudades que pertenecen al mismo cluster
    cities_in_cluster = df_train[df_train['Cluster'] == cluster_id]
    
    if cities_in_cluster.empty:
        closest_cities.append(None)
        continue
    
    # Escalar las ciudades de ese cluster (usando el mismo scaler)
    cities_in_cluster_scaled = scaler.transform(cities_in_cluster.drop(['City', 'Cluster'], axis=1))
    
    # Acceder a la fila escalada de df_test_scaled usando `idx`
    test_city_scaled = df_test_scaled[idx].reshape(1, -1)
    
    # Calcular la distancia de la ciudad de prueba a las ciudades del cluster
    distances_to_cities = cdist(test_city_scaled, cities_in_cluster_scaled, 'euclidean')
    
    # Encontrar el índice de la ciudad más cercana dentro del cluster
    closest_city_index = np.argmin(distances_to_cities)
    
    # Obtener el nombre de la ciudad más cercana
    closest_city = cities_in_cluster.iloc[closest_city_index]['City']
    
    closest_cities.append(closest_city)

# Añadir las ciudades más cercanas usando .loc
df_test.loc[:, 'Closest_City'] = closest_cities
# Crear archivo de submission
df_submission = df_test[['City', 'Closest_City']]
df_submission.columns = ['ID', 'label']
df_submission.to_csv('resultado.csv', index=False)
